# Parse development notebook

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

### Prerequisities
 * generated json schema for all used objects (run generate-json-schemas.py)

### JSon schemas for the actants
...
...

### The import tables:
 * Texts
 * Manuscripts (must be done alongside T, David thinks) = O of class defined by col. class_id
 * Resources
 * C
 * A

### Input variables

In [1]:
#                  sheet_name,  code, header_in_row
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 4),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 4),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 4), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 4) # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with scheas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

### Libraries

In [2]:
import os, warlock, json
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv
import pandas as pd
import numpy as np


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [3]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-02-15 18:33:02 : Google authentification start
20 2022-02-15 18:33:02 : Google authentification end
20 2022-02-15 18:33:02 : Dator initiation succesfull end


In [4]:
# read all schemas inside and warlock them
schema_filenames = os.listdir(schema_path)
for schema in schema_filenames:
    name = schema.split(".")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    logger.info("Class " + name + " available.")

2022-02-15 18:33:02,269 INFO Class IActant available.
2022-02-15 18:33:02,279 INFO Class IAction available.
2022-02-15 18:33:02,287 INFO Class IEntity available.
2022-02-15 18:33:02,295 INFO Class ILabel available.
2022-02-15 18:33:02,303 INFO Class IProp available.
2022-02-15 18:33:02,312 INFO Class IResource available.
2022-02-15 18:33:02,321 INFO Class IStatement available.
2022-02-15 18:33:02,329 INFO Class ITerritory available.
2022-02-15 18:33:02,330 INFO Class IUser available.


In [12]:
# load all input tables
tables = {}
header_infos = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=True, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True)

2022-02-15 18:33:22,566 INFO Calling for texts with sheet_name Texts.


20 2022-02-15 18:33:23 : Loading dataset Texts
20 2022-02-15 18:33:23 : Opting for variant header at row 5.
20 2022-02-15 18:33:28 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-02-15 18:33:28 : Dropping empty columns in the dataset Texts : (1011, 92)
20 2022-02-15 18:33:28 : Deleted 869 empty rows by label.
20 2022-02-15 18:33:28 : Loaded and prepared dataset Texts : (142, 92)


2022-02-15 18:33:28,584 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-02-15 18:33:28 : Making pickle cache of  Texts with separeted header file : (142, 92)
20 2022-02-15 18:33:28 : Loading dataset Manuscripts
20 2022-02-15 18:33:28 : Opting for variant header at row 4.


2022-02-15 18:33:31,686 INFO Calling for resources with sheet_name Resources.


20 2022-02-15 18:33:31 : Hyperlinks were detected and transformed in columns ['reproduction_online_url', 'reproduction_note'].
20 2022-02-15 18:33:31 : Dropping empty columns in the dataset Manuscripts : (999, 43)
20 2022-02-15 18:33:31 : Deleted 860 empty rows by label.
20 2022-02-15 18:33:31 : Loaded and prepared dataset Manuscripts : (139, 43)
20 2022-02-15 18:33:31 : Making pickle cache of  Manuscripts with separeted header file : (139, 43)
20 2022-02-15 18:33:31 : Loading dataset Resources
20 2022-02-15 18:33:31 : Opting for variant header at row 4.


2022-02-15 18:33:34,370 INFO Calling for actions with sheet_name Statements.


20 2022-02-15 18:33:34 : Hyperlinks were detected and transformed in columns [].
20 2022-02-15 18:33:34 : Dropping empty columns in the dataset Resources : (1000, 20)
20 2022-02-15 18:33:34 : Deleted 934 empty rows by label.
20 2022-02-15 18:33:34 : Loaded and prepared dataset Resources : (66, 20)
20 2022-02-15 18:33:34 : Making pickle cache of  Resources with separeted header file : (66, 20)
20 2022-02-15 18:33:34 : Loading dataset Statements
20 2022-02-15 18:33:34 : Opting for variant header at row 4.


2022-02-15 18:33:41,291 INFO Calling for concepts with sheet_name Concepts.


20 2022-02-15 18:33:41 : Hyperlinks were detected and transformed in columns [].
20 2022-02-15 18:33:41 : Dropping empty columns in the dataset Statements : (1030, 73)
20 2022-02-15 18:33:41 : Deleted 588 empty rows by label.
20 2022-02-15 18:33:41 : Loaded and prepared dataset Statements : (442, 73)
20 2022-02-15 18:33:41 : Making pickle cache of  Statements with separeted header file : (442, 73)
20 2022-02-15 18:33:42 : Loading dataset Concepts
20 2022-02-15 18:33:42 : Opting for variant header at row 4.
20 2022-02-15 18:33:46 : Hyperlinks were detected and transformed in columns [].
20 2022-02-15 18:33:46 : Dropping empty columns in the dataset Concepts : (3019, 57)
20 2022-02-15 18:33:46 : Deleted 724 empty rows by label.
20 2022-02-15 18:33:46 : Loaded and prepared dataset Concepts : (2295, 57)
20 2022-02-15 18:33:46 : Making pickle cache of  Concepts with separeted header file : (2295, 57)


In [13]:
tables['texts']

,id,label,language,label_short,text_name_original,detail,region_covered,microregion_covered,author_label,language_id,...,dissinet_coding_priority,dissinet_person,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,parsing_rows_explained
0,T1,Process against Bernard Niort and his family,English,NaN,NaN,Early 1234.,Languedoc,NaN,NaN,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposition,NaN,NaN
1,T2,Sentences of William Arnold and Stephen of Sai...,English,NaN,NaN,NaN,Languedoc,Toulousain #Lauragais,William Arnold #Stephen of Saint-Thibéry,C0938,...,NaN,RS?,NaN,NaN,NaN,NaN,register,sentence,End-folio sometimes cited as 184v (e.g. Roche...,NaN
2,T3,Peter Seila’s Register of Penances,English,Seila,Penitenciae fratris Petri Sellani,Penitenciae fratris Petri Sellani. Register of...,Languedoc,Quercy (west),NaN,C0938,...,1,RS,NaN,NaN,NaN,NaN,register,sentence #culpa,NaN,NaN
3,T4,Register FFF of the Carcassonne inquisition,English,FFF,NaN,NaN,Languedoc,Montségur #Lauragais #Cabardès #Quercy (east) ...,Ferrer #William Raymond #Pons Gary #Peter Durand,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,register,deposition,NaN,NaN
4,T5,Confirmation of depositions before Ferrer and ...,English,NaN,NaN,NaN,Languedoc,NaN,Ferrer #Pons Gary,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,register,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,T139,letter of Evervin of Steinfeld to Bernard of C...,English,NaN,NaN,NaN,NaN,NaN,NaN,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,T140,letter from Liège to pope,English,NaN,NaN,NaN,NaN,NaN,NaN,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,T141,Annales Aquenses,Latin,NaN,NaN,NaN,NaN,NaN,NaN,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,T142,Annales Rodenses,Latin,NaN,NaN,NaN,NaN,NaN,NaN,C0938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
header_infos['texts']

,id,label,language,label_short,text_name_original,detail,region_covered,microregion_covered,author_label,language_id,...,dissinet_coding_priority,dissinet_person,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,parsing_rows_explained
0,,,,,Archive - now concatenated with detail. Would ...,,,,We will easily recreate this in InkV - not wor...,,...,,,,,,,,,,Comment - sometimes important to read for pars...
1,,,,,,,,,,,...,,,,,,,,,,Source node (for props)
2,,,,,,,,,,C0732,...,,,,,,,,,,Prop type (for props) or further detail (for i...
3,inside,inside,inside,discard,discard,inside,??? discard,discard,discard,prop,...,discard,discard,discard,discard,discard,discard,discard,discard,inside,discard


#### Territories parsing instruction
there is info column : parsing_rows_explained, comments on the rows

row 0 : human comment
row 1 : source node for props
row 2 : prop type for props or further details
row 3 : parse keyword (inside, discard, prop, prop: ..., special)

possibilities by parse "keyword" (in the last row = 4)
 * inside
 * discard
 * prop in row 4 +  concept id inrow  3
 * prop with ':' value and other inforomation > taken provisionally as discard
 * special > completely custom behavior in row 0
 * ???, ??? discard  > taken as discard

In [8]:
class Parser():

    def __init__(self, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger:logger):
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],'prop':[]}
        self.logger = logger


    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = keyword_row.at[c]
            prop_type_candidate = prop_type_row.at[c]
            source_node_candidate = source_node_row.at[c]

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)
            elif 'prop' in instruction_candidate:
                # TODO validation
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'prop', 'type': prop_type, 'source':source_node}
                self.oper_columns['prop'].append(c)
            elif 'special' in instruction_candidate:
                # TODO will look for custom functions registered by column name
                instruction  = {'operation':'special', 'target': None}
                self.oper_columns['special'].append(c)
            elif 'inside' in instruction_candidate:
                instruction  = {'operation':'inside', 'target': None}
                self.oper_columns['inside'].append(c)
            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction
        self.logger.info(f"Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()
        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)
        self.logger.info(f" {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},prop:{len(self.oper_columns['prop'])}, special:{len(self.oper_columns['special'])}. Originally {self.input_table_df.shape[1]} columns.")
        return ip

    def prepare_property(self):
        pass

    def process_items(self):

        # for each table row

            # take column value and
                # if inside
                    # map it to the entity object



        pass


In [9]:
# texts are "root" territories, they are territories without parent

texts_parser = Parser(header_infos['texts'], tables['texts'], 3, d.logger)
texts_parser.process_header_instructions()
texts_parser.prepare_input_table()

2022-02-15 18:33:02,530 INFO Uncertain parsing instructions in 22 columns: REGION_COVERED:??? discard,, TIMERELATION1_TYPE:???,, TIMERELATION1_TARGET_ID:???,, TIMERELATION2_TYPE:???,, TIMERELATION2_TARGET_ID:???,, TIMERELATION3_TYPE:???,, TIMERELATION3_TARGET_ID:???,, TIMERELATION4_TYPE:???,, TIMERELATION4_TARGET_ID:???,, MANUSCRIPT_WITNESS_1_WITH_RANGE:prop: manuscript_witness_1,???C page range, MANUSCRIPT_WITNESS_2_WITH_RANGE:???,, MANUSCRIPT_WITNESS_3_WITH_RANGE:???,, MANUSCRIPT_WITNESS_4_WITH_RANGE:???,, MANUSCRIPT_WITNESS_5_WITH_RANGE:???,, MANUSCRIPT_WITNESS_6_WITH_RANGE:???,, MANUSCRIPT_WITNESS_7_WITH_RANGE:???,, MANUSCRIPT_WITNESS_8_WITH_RANGE:???,, MANUSCRIPT_WITNESS_9_WITH_RANGE:???,, MANUSCRIPT_WITNESS_10_WITH_RANGE:???,, MANUSCRIPT_WITNESS_11_WITH_RANGE:???,, EDITION_1_PAGE_RANGE:???,, EDITION_1_NOTE:???,,.
2022-02-15 18:33:02,533 INFO  68 columns have been dropped (discard:44, unknown:24). Table now has 24 columns, inside:5,prop:16, special:3. Originally 92 columns.


,id,label,language,detail,language_id,genre_id,milieu_of_provenance_id,origin_id,manuscript_witness_1,manuscript_witness_1_with_range,...,manuscript_witness_6,manuscript_witness_7,manuscript_witness_8,manuscript_witness_9,manuscript_witness_10,manuscript_witness_11,edition_1,edition_2,edition_3,note
0,T1,Process against Bernard Niort and his family,English,Early 1234.,C0938,C1134,C1138,NaN,M21,21: 34r-50r,...,NaN,NaN,NaN,NaN,NaN,NaN,"Douais, 1900",NaN,NaN,NaN
1,T2,Sentences of William Arnold and Stephen of Sai...,English,NaN,C0938,C1135,C1138,NaN,M21,21: 143v-185r,...,NaN,NaN,NaN,NaN,NaN,NaN,NA,NaN,NaN,End-folio sometimes cited as 184v (e.g. Roche...
2,T3,Peter Seila’s Register of Penances,English,Penitenciae fratris Petri Sellani. Register of...,C0938,C1135#C1136,C1138,L0011,M21,21: 185r-312v,...,NaN,NaN,NaN,NaN,NaN,NaN,"Duvernoy, 2001",NaN,NaN,NaN
3,T4,Register FFF of the Carcassonne inquisition,English,NaN,C0938,C1134,C1138,L0006,M22 #M23 #M24,22: 107r-296v #23:1r-346v #24:1r-238r,...,NaN,NaN,NaN,NaN,NaN,NaN,"Duvernoy, 1998","Blaquière & Dossat, 1968",NaN,NaN
4,T5,Confirmation of depositions before Ferrer and ...,English,NaN,C0938,C1122,C1138,NaN,M4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"Duvernoy, 1983",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,T139,letter of Evervin of Steinfeld to Bernard of C...,English,NaN,C0938,C1129,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,T140,letter from Liège to pope,English,NaN,C0938,C1129,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,T141,Annales Aquenses,Latin,NaN,C0938,C1364,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,T142,Annales Rodenses,Latin,NaN,C0938,C1364,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
manuscripts_parser = Parser(header_infos['manuscripts'], tables['manuscripts'], 2, d.logger)
manuscripts_parser.process_header_instructions()
manuscripts_parser.prepare_input_table()

2022-02-15 18:33:02,576 INFO Uncertain parsing instructions in 3 columns: COMPONENT_ID:prop,???, REPRODUCTION_ONLINE_URL:???,,This will be R. REPRODUCTION_AVAILABLE_IN_BRNO:discard??? Clash with R representing repros,,.
2022-02-15 18:33:02,579 INFO  30 columns have been dropped (discard:16, unknown:14). Table now has 13 columns, inside:4,prop:9, special:0. Originally 43 columns.


,id,label,detail,language,class_id,component_id,shelf_number,alternative_shelf_number,catalogue_number,height_cm,width_cm,folios_total,material_id
0,M106,"Bologna, Biblioteca comunale dell’Archiginnasi...",NaN,NaN,C1184,NaN,ms. B 1856,16.gg.I.1,NaN,38.5,28.5,161,NaN
1,M2,"London, British Library, Add MS 4697",Gui Sentences,NaN,C1184,NaN,Add MS 4697,Plut.I.F,NaN,36,25,221,NaN
2,M102,"Vaticano, Archivio Segreto Vaticano, Camera ap...",NaN,NaN,C1184,NaN,Collectoria 133,NaN,NaN,NaN,NaN,NaN,NaN
3,M104,"Modena, Archivio di Stato di Modena, Bibliotec...",NaN,NaN,C1184,NaN,ms. 132,NaN,NaN,NaN,NaN,NaN,NaN
4,M4,"Carcassonne, Archives départementales de l’Aud...",NaN,NaN,NaN,NaN,ms. 3 J 596,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,M153,"St Paul-im-Lavanttal, Stift Sankt Paul-im-Lava...",NaN,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,NaN,NaN
135,M154,"Roma, Biblioteca Casanatense, ms. lat. 3217",NaN,NaN,NaN,NaN,ms. lat. 3217,D.III.18,NaN,NaN,NaN,NaN,NaN
136,M155,", Archivio Generale dell'Ordine Domenicano, II.64",NaN,NaN,NaN,NaN,II.64,NaN,NaN,NaN,NaN,NaN,NaN
137,M156,"Wien, Österreichische Nationalbibliothek, Cod....",NaN,NaN,NaN,NaN,Cod. 1137,NaN,NaN,18,11.5,160,NaN


In [14]:
test1 = ITerritory({'class':"T",
                  'data': {'parent':False},
                  'detail': "",
                  'id':"",
                  'label':"",
                  'language':"",
                  'notes':[],
                  'props':[],
                  'status':"0"})
#test2 = IActant()

In [15]:
test1

{'class': 'T',
 'data': {'parent': False},
 'detail': '',
 'id': '',
 'label': '',
 'language': '',
 'notes': [],
 'props': [],
 'status': '0'}

**Goal:**  to have a json file full of territories, which will be added to the mock up actants.json file and inserted to RethinkDb and fully visisble, accessible in inkVisitor.

Steps
 * study the mockup actants.json structure  DONE
 * prepare a code intepreting and operatinalizing David's import instruction in the header_df  DONE
 * parse the table to the territories.json
   * helper class for "territory", which fills default values
   *
 * merge json files
 * import ...

Forseen complexities
 * '#' hash notation, so called *multiples*, in the id fields, C2015#C0156. ...
 * '~' tilda notation
 * urls in cell (formula =HYPERLINK), SOLVED in Dator class

